In [1]:
import numpy as np
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
from pyiomica_master import pyiomica as pio
from pyiomica_master.pyiomica import visualizationFunctions
from pyiomica_master.pyiomica import visibilityGraphCommunityDetection
import matplotlib as mpl
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from tqdm import tqdm
%matplotlib inline

Loading PyIOmica 1.3.3 (https://github.com/gmiaslab/pyiomica by G. Mias Lab)


/data/user/home/sahaj432/Desktop/code_organized/G_signal_feature_reduction_Project/New_neural_code_SNR_2_v/pyiomica_master/pyiomica/visibilityGraphAuxiliaryFunctions.py:91: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(cache=True)


In [2]:
x_frame1_05 = np.genfromtxt('/home/sahaj432/Desktop/dataset/neural_4_class/x_frame1_2_seed.csv', delimiter=',')
x_frame2_05 = np.genfromtxt('/home/sahaj432/Desktop/dataset/neural_4_class/x_frame2_2_seed.csv', delimiter=',')
x_frame3_05 = np.genfromtxt('/home/sahaj432/Desktop/dataset/neural_4_class/x_frame3_2_seed.csv', delimiter=',')
x_framen_05= np.genfromtxt('/home/sahaj432/Desktop/dataset/neural_4_class/x_frame_noise_2_seed.csv', delimiter=',')

In [3]:
x=np.concatenate((x_frame1_05,x_frame2_05,x_frame3_05,x_framen_05),axis=0)
y=np.concatenate((np.zeros(len(x_frame1_05)),np.zeros(len(x_frame2_05))+1,np.zeros(len(x_frame3_05))+2,np.zeros(len(x_framen_05))+3),axis=0)

In [4]:
X, Y = shuffle(x,y, random_state=1)

In [5]:
def calculate_adj(data):
    temp1=[]
    for p, i in enumerate(data):
        #print(f'Converting ==> {p+1} \r', end="",flush=True)
        vs_out = visibilityGraphCommunityDetection.createVisibilityGraph(i, range(len(i)), "natural", weight='tan')
        temp1.append(vs_out)
        
    return temp1

In [6]:
from multiprocessing import Pool
import multiprocessing

if __name__ == '__main__':
    num_cores = multiprocessing.cpu_count()
    print("Number of CPU cores:", num_cores)

Number of CPU cores: 48


In [7]:
df=np.array_split(X, num_cores)
print(len(df[0]))

642


In [8]:
with Pool(processes=num_cores) as pool:
    results = pool.map(calculate_adj, df)

pool.close()
pool.join()

In [9]:
x_adj=[]
for submatrix in results:
    x_adj.extend(submatrix)

In [10]:
x_adj=np.array(x_adj)

In [11]:
def calculate_g(data):
    temp2=[]
    for vs_out in data:
        a = np.sum(vs_out, axis=0).reshape(-1, 1)
        b = np.sum(vs_out != 0, axis=1).reshape(-1, 1)
        temp2.append(np.concatenate((a, b), axis=1))
        
    return temp2

In [12]:
from multiprocessing import Pool
import multiprocessing

if __name__ == '__main__':
    num_cores = multiprocessing.cpu_count()
    print("Number of CPU cores:", num_cores)


df1=np.array_split(x_adj[:], num_cores)


print(len(df[0]))




Number of CPU cores: 48
642


In [13]:

import time
start_time = time.time()
with Pool(processes=num_cores) as pool1:
    results1 = pool1.map(calculate_g, df1)

pool1.close()
pool1.join()

print("--- %s seconds ---" % (time.time() - start_time))


--- 1.5706143379211426 seconds ---


In [14]:
x_g=[]
for submatrix in results1:
    x_g.extend(submatrix)

In [15]:
x_g=np.array(x_g)


In [16]:
print(x_g.shape)

(30783, 56, 2)


In [17]:
x_g_flat=np.array([np.array(i.T.flatten()).ravel() for i in x_g])


In [18]:
x_train, x_temp, y_train, y_temp = train_test_split(x_g_flat, Y, test_size=0.30, random_state=42)
x_val,x_test,y_val,y_test=train_test_split(x_temp,y_temp,test_size=0.50, random_state=42)

In [19]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(21548, 112)
(4617, 112)
(4618, 112)


In [20]:
import tensorflow as tf


2023-08-13 09:49:12.700830: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/18.08.9/lib64/slurm:/cm/shared/apps/slurm/18.08.9/lib64
2023-08-13 09:49:12.700860: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [21]:
input_shape = (x_train.shape[1],1)
num_classes = 4
model3 = tf.keras.models.Sequential()
model3.add(tf.keras.layers.Conv1D(filters= 32, kernel_size=3, activation='relu',padding='same',input_shape= input_shape))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))

model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))

# model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
# model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
# model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
# model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
# model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
# model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
# model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
# model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
model3.add(tf.keras.layers.Flatten())

model3.add(tf.keras.layers.Dense(100, activation='relu'))
model3.add(tf.keras.layers.Dropout(0.1))
model3.add(tf.keras.layers.Dense(100, activation='relu'))
model3.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

2023-08-13 09:49:32.782974: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/18.08.9/lib64/slurm:/cm/shared/apps/slurm/18.08.9/lib64
2023-08-13 09:49:32.783003: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-08-13 09:49:32.783022: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c0186): /proc/driver/nvidia/version does not exist
2023-08-13 09:49:32.783322: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
model3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 112, 32)           128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 56, 32)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 56, 32)            3104      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 28, 32)           0         
 1D)                                                             
                                                                 
 flatten (Flatten)           (None, 896)               0         
                                                                 
 dense (Dense)               (None, 100)               8

In [23]:
model3.compile(loss='SparseCategoricalCrossentropy', optimizer='Adam', metrics=['acc'])


In [24]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model



checkpoint = ModelCheckpoint('F8.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')


In [25]:
history_log = model3.fit(x_train, y_train, epochs=30, verbose=1, validation_data=(x_test, y_test), callbacks=[checkpoint])


Epoch 1/30
674/674 [==============================] - 3s 4ms/step - loss: 0.4554 - acc: 0.8130 - val_loss: 0.2754 - val_acc: 0.8919
Epoch 2/30
674/674 [==============================] - 2s 3ms/step - loss: 0.2750 - acc: 0.8940 - val_loss: 0.2640 - val_acc: 0.8987
Epoch 3/30
674/674 [==============================] - 2s 3ms/step - loss: 0.2547 - acc: 0.9005 - val_loss: 0.2482 - val_acc: 0.9056
Epoch 4/30
674/674 [==============================] - 2s 4ms/step - loss: 0.2450 - acc: 0.9050 - val_loss: 0.2500 - val_acc: 0.9036
Epoch 5/30
674/674 [==============================] - 2s 3ms/step - loss: 0.2354 - acc: 0.9077 - val_loss: 0.2516 - val_acc: 0.9006
Epoch 6/30
674/674 [==============================] - 2s 3ms/step - loss: 0.2268 - acc: 0.9124 - val_loss: 0.2331 - val_acc: 0.9123
Epoch 7/30
674/674 [==============================] - 2s 3ms/step - loss: 0.2191 - acc: 0.9164 - val_loss: 0.2510 - val_acc: 0.9028
Epoch 8/30
674/674 [==============================] - 2s 3ms/step - loss: 0.

Epoch 29/30
674/674 [==============================] - 2s 3ms/step - loss: 0.0658 - acc: 0.9746 - val_loss: 0.4274 - val_acc: 0.8965
Epoch 30/30
674/674 [==============================] - 2s 3ms/step - loss: 0.0683 - acc: 0.9750 - val_loss: 0.4581 - val_acc: 0.8945


In [26]:
model3.save_weights('/home/sahaj432/Desktop/code_organized/G_signal_feature_reduction_Project/New_neural_code_SNR_2_v/F8.h5')


In [27]:
model3.load_weights('/home/sahaj432/Desktop/code_organized/G_signal_feature_reduction_Project/New_neural_code_SNR_2_v/F8.h5')


In [28]:
predictions = model3.predict(x_test)
pred_arg = np.argmax(predictions, axis=1)

In [29]:
from sklearn.metrics import confusion_matrix
cnf_matrix=confusion_matrix(y_test,pred_arg)
print("cnf_matrix_2.0_70")
print("")
print(cnf_matrix)

cnf_matrix_2.0_70

[[ 807    3    0   73]
 [   0  816  107   59]
 [   1  130  797    0]
 [  44   68    2 1711]]


In [30]:
from sklearn.metrics import accuracy_score, recall_score,precision_score,roc_auc_score,roc_curve,f1_score


In [31]:
print(accuracy_score(y_test,pred_arg)*100)
print(f1_score(y_test,pred_arg,average='macro')*100)
print(recall_score(y_test,pred_arg, average="weighted")*100)
print(precision_score(y_test,pred_arg, average="weighted")*100)

89.45430922477263
88.71848859809965
89.45430922477263
89.53924823422577
